In [2]:
import pandas as pd
import numpy as np
import holidays

In [3]:
file_path = '/Users/mymac/Google Drive/My Drive/Forex_Robot/'

In [3]:
# Get the data
def percentage_to_float(x):
  return float(x.strip('%')) / 100
  
us_holidays = holidays.US()

df = pd.read_csv(file_path + 'chris.csv')
df.columns = map(str.lower, df.columns)
df.drop(['simple date', 'day', 'day type name', 'holiday flag', 'date.1'], axis=1, inplace=True)
df.date = pd.to_datetime(df.date)
df.reset_index(drop=True, inplace=True)
df.fillna(0, inplace=True)
df['20 day rel vol %'] = df['20 day rel vol %'].astype(str)
df['20 day rel vol %'] = df['20 day rel vol %'].apply(lambda x: percentage_to_float(x))
daytype_onehot = pd.get_dummies(df.daytype, prefix='daytype')
df.drop('daytype', axis=1, inplace=True)
df = pd.concat([df, daytype_onehot], axis=1)
df['holiday'] = df['date'].apply(lambda x: 1 if x.strftime('%Y-%m-%d') in us_holidays else 0)

df

,date,above 100 ema?,above 50 ema?,ema's aligned?,100 ema,50 ema,5 ema,ticks from 5ema,distance from 5ema to 50ema,% distance of 5ema to 50ema,...,ticks down at rth close,open to on lo on up on sessions (ticks),open to on high on down on sessions (ticks),daytype_1,daytype_2,daytype_3,daytype_4,daytype_5,daytype_6,holiday
0,2012-01-05 14:00:00,0,0,0,1110.00,1110.00,1110.00,0,0.00,0.00%,...,0,22,0,0,1,0,0,0,0,0
1,2012-01-06 14:00:00,0,0,0,1110.50,1110.50,1110.50,2,0.00,0.00%,...,0,30,0,0,0,1,0,0,0,0
2,2012-01-09 14:00:00,1,1,0,1110.75,1110.75,1110.75,4,0.00,0.00%,...,0,3,0,1,0,0,0,0,0,0
3,2012-01-10 14:00:00,0,0,1,1114.00,1114.00,1114.50,30,0.50,0.04%,...,7,0,13,1,0,0,0,0,0,0
4,2012-01-11 14:00:00,1,1,1,1116.75,1116.75,1118.25,28,1.50,0.13%,...,0,13,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372,2021-03-15 14:00:00,1,1,1,3712.75,3821.25,3917.00,159,95.75,2.51%,...,0,27,0,0,1,0,0,0,0,0
2373,2021-03-16 14:00:00,1,1,1,3717.75,3826.75,3930.50,108,103.75,2.71%,...,79,0,12,1,0,0,0,0,0,0
2374,2021-03-17 14:00:00,1,1,1,3722.50,3832.00,3940.75,82,108.75,2.84%,...,103,0,66,0,1,0,0,0,0,0
2375,2021-03-18 14:00:00,1,1,1,3726.25,3835.00,3930.75,-81,95.75,2.50%,...,34,0,51,0,1,0,0,0,0,0


In [4]:
df.columns

Index(['date', 'above 100 ema?', 'above 50 ema?', 'ema's aligned?', '100 ema',
       '50 ema', '5 ema', 'ticks from 5ema', 'distance from 5ema to 50ema',
       '% distance of 5ema to 50ema', 'distance from 5ema to 100ema',
       '% distance of 5ema to 100ema', 'distance from 50ema to 100ema',
       '% distance of 50ema to 100ema', '200 sma', '5 day atr', '10 day atr',
       'delta close', 'delta hi', 'delta lo', 'open', 'on hi', 'on lo',
       'rth open', 'high', 'low', 'close / rth close', 'ticks up or down',
       '% gain on the day', 'green day?', 'range in ticks',
       'range as a % of price', 'top 25% close', 'lower 25% close',
       'mid close', 'monster day up?', 'monster day dn?', 'higher low?',
       '1tf up?', 'consecutive 1tf up days', 'lower high?', '1tf down?',
       'consecutive 1tf up days.1', 'volume', '20 day rel vol %', 'on open',
       'on hi.1', 'on lo.1', 'rth open.1', 'beob', 'buob', 'bull hammer',
       'bear hammer', 'bullish idf', 'bearish idf', '

In [7]:
# Run simulation
reward = 0
n_wins = 0
n_losses = 0
win_streak = 0
loss_streak = 0
curr_win_streak = 0
curr_loss_streak = 0
n_buys = 0
n_sells = 0
n_ticks = 0
trade = None
risk = 175 / 4

for i in range(1, df.shape[0]):
  on_open = df.loc[df.index[i], 'on open']
  rth_open = df.loc[df.index[i - 1], 'rth open']
  rth_close = df.loc[df.index[i - 1], 'close / rth close']
  ema_5, ema_50, ema_100 = df.loc[df.index[i - 1], ['5 ema', '50 ema', '100 ema']]
  vix = df.loc[df.index[i - 1], 'vix adj close']

  down_day = rth_close < rth_open
  ema_signal = ema_5 < ema_50 or ema_5 < ema_100
  vix_signal = vix >= 16

  trade_signal = down_day and ema_signal and vix_signal

  if trade_signal and trade is None:
    open_price = on_open
    stop_loss = on_open + risk

    trade = {'open_price': open_price, 'stop_loss': stop_loss, 'trade_type': 'sell'}

    n_sells += 1

  if trade is not None:
    on_high = df.loc[df.index[i], 'on hi']
    on_low = df.loc[df.index[i], 'on lo']
    on_close = df.loc[df.index[i], 'rth open']

    if trade['trade_type'] == 'buy':
      if on_low <= trade['stop_loss']:
        trade_amount = trade['stop_loss'] - trade['open_price']
        reward += trade_amount
        n_ticks += trade_amount * 4

        n_wins += 1 if trade_amount > 0 else 0
        n_losses += 1 if trade_amount < 0 else 0

        trade = None

        continue

      else:
        trade_amount = on_close - trade['open_price']
        reward += trade_amount
        n_ticks += trade_amount * 4

        n_wins += 1 if trade_amount > 0 else 0
        n_losses += 1 if trade_amount < 0 else 0

        trade = None

        continue

    else:
      if on_high >= trade['stop_loss']:
        trade_amount = trade['open_price'] - trade['stop_loss']
        reward += trade_amount
        n_ticks += trade_amount * 4

        n_wins += 1 if trade_amount > 0 else 0
        n_losses += 1 if trade_amount < 0 else 0

        trade = None

        continue

      else:
        trade_amount = trade['open_price'] - on_close
        reward += trade_amount
        n_ticks += trade_amount * 4

        n_wins += 1 if trade_amount > 0 else 0
        n_losses += 1 if trade_amount < 0 else 0

        trade = None

        continue

In [8]:
print(f'Reward: {reward}\nTicks: {n_ticks}\nBuys: {n_buys}\nSells: {n_sells}\nTrades: {n_buys + n_sells}\nWins: {n_wins}\nLosses: {n_losses}\nWin rate: {n_wins / (n_wins + n_losses)}')

Reward: 165.0
Ticks: 660.0
Buys: 0
Sells: 234
Trades: 234
Wins: 122
Losses: 110
Win rate: 0.5258620689655172
